<a href="https://colab.research.google.com/github/johnnyff/bigcontest2021/blob/main/danm_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm

import datetime as dt
import torch.nn as nn

warnings.filterwarnings(action='ignore') 

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/bigcontest/data/bigcontest_dataset.xlsx', header = 1)
df

In [ ]:
columns = ['num','year','month','day','hour','target',
           'average_rain_1','a_rain_1','b_rain_1','c_rain_1','d_rain_1','e_level_1','d_level_1',
           'average_rain_2','a_rain_2','b_rain_2','c_rain_2','d_rain_2','e_level_2','d_level_2',
           'average_rain_3','a_rain_3','b_rain_3','c_rain_3','d_rain_3','e_level_3','d_level_3',
           'average_rain_4','a_rain_4','b_rain_4','c_rain_4','d_rain_4','e_level_4','d_level_4',
           'average_rain_5','a_rain_5','b_rain_5','c_rain_5','d_rain_5','e_level_5','d_level_5',
           'average_rain_6','a_rain_6','b_rain_6','c_rain_6','d_rain_6','e_level_6','d_level_6'
           ]

In [ ]:
df.columns =columns


In [ ]:
df.drop(['num','year','month','day','hour'],axis =1 , inplace = True)
df

In [ ]:
train_data = df[:-160]
train_data

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.duca.manual_seed_all(777)

In [ ]:
import torch
from torch.nn import Module, LSTM, Linear
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable

In [ ]:
sc = MinMaxScaler()
training_data = sc.fit_transform(train_data)


In [ ]:
training_data.shape

In [ ]:
torch.FloatTensor(training_data).view(-1)

In [ ]:
training_data.shape

In [ ]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length][0]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [ ]:



sc = MinMaxScaler()
training_data = sc.fit_transform(train_data)

seq_length = 5
x, y = sliding_windows(training_data, seq_length)

train_size = int(len(y) * 0.7)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(x))
dataY = Variable(torch.Tensor(y))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

In [ ]:
print(x.shape)
print(y.shape)
print(x[:6])
print(y[0])

In [ ]:
class LSTM(nn.Module):

    def __init__(self):
        super(LSTM, self).__init__()
       
        self.lstm = nn.LSTM(input_size=43, hidden_size=128,
                            num_layers=2, batch_first=True)
        
        self.fc = nn.Linear(128, 1)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        # Propagate input through LSTM
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x 

In [ ]:
num_epochs = 100
learning_rate = 0.01

input_size = 43
hidden_size = 2
num_layers = 2

num_classes = 1

net = LSTM().to(device)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    var_x = Variable(trainX).to(device)
    var_y = Variable(trainY).to(device)
    
    out = net(var_x)
    # obtain the loss function
    loss = criterion(out, var_y)
    
    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()
    if epoch % 10 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

In [ ]:
net = net.eval() 

In [ ]:
with torch.no_grad():
  var_data = Variable(testX)
  pred_test = net(var_data)

In [ ]:
cp = np.repeat(pred_test, 43, axis = -1)

In [ ]:
sc.inverse_transform(pred_test.squeeze())

In [ ]:
pred_test.squeeze().shape